# Openflow SPCS Setup Guide

This notebook guides you through setting up Snowflake Openflow on Snowpark Container Services (SPCS). Openflow enables real-time data integration and streaming analytics directly within Snowflake.

## Important Notes
- **Preview Feature**: Openflow on SPCS is rolling out in preview to all accounts in AWS and Azure Commercial Regions from September 17, 2025
- **Account Rollout**: Snowflake is gradually rolling out this feature. Not all accounts may have access immediately
- **Compatibility**: This notebook is compliant with the September 2025 preview release of Openflow SPCS

## Setup Overview

This notebook follows the official Snowflake documentation and covers four main steps:

1. **[Core Snowflake Setup](https://docs.snowflake.com/en/user-guide/data-integration/openflow/setup-openflow-spcs-sf)** - Configure roles, privileges, and network rules
2. **[Create Deployment](https://docs.snowflake.com/en/user-guide/data-integration/openflow/setup-openflow-spcs-deployment)** - Set up the Openflow deployment environment
3. **[Create Runtime Role](https://docs.snowflake.com/en/user-guide/data-integration/openflow/setup-openflow-spcs-create-rr)** - Configure runtime permissions
4. **[Create Runtime](https://docs.snowflake.com/en/user-guide/data-integration/openflow/setup-openflow-spcs-create-runtime)** - Deploy the runtime environment

## Prerequisites
- `ACCOUNTADMIN` role or equivalent privileges
- Access to Snowsight (Snowflake web interface)
- Account in AWS or Azure Commercial Regions
- Understanding of your network security requirements


In [ ]:
# Configuration - Update these values for your environment
# ==========================================================

# Set Implementation Role
IMPLEMENTATION_ROLE = "ACCOUNTADMIN"

# Context where objects are created
DATABASE = "OPENFLOW"
SCHEMA = "OPENFLOW"

# User Configuration
OPENFLOW_USER = "DCHAFFELSON"  # Update with the user that will access Openflow

# Warehouse Configuration
WAREHOUSE_NAME = "OPENFLOW_WH"  # Update to your warehouse name

# Network Configuration (from ingressPersistenceTask.ipynb)
INGRESS_NETWORK_RULE_NAME = "ALLOW_OPENFLOW_DEPLOYMENT_INGRESS"
SNOWPIPE_STREAMING_NETWORK_RULE_NAME = "OPENFLOW_SNOWPIPE_STREAMING_NETWORK_RULE"
SNOWPIPE_STREAMING_EAI_NAME = "OPENFLOW_SNOWPIPE_STREAMING_EAI"
IP_ADDRESS_LIST = "('10.16.0.0/12')"  # SPCS range + your BYOC deployment IP if you have one
NETWORK_POLICY_NAME = "ACCOUNT_VPN_POLICY_SE"  # Update if you have account-level network policy

# Openflow Deployment Configuration
DEPLOYMENT_NAME = "SPCS1"
EVENT_TABLE_NAME = "EVENTS"

# Runtime Configuration
ADMIN_ROLE_NAME = "OPENFLOWADMIN"
RUNTIME_NAME = DEPLOYMENT_NAME + "_RUNTIME1"
RUNTIME_ROLE_NAME = "OPENFLOWRUNTIMEROLE_" + RUNTIME_NAME

print("✓ Configuration variables set successfully!")
print(f"Database: {DATABASE}")
print(f"Schema: {SCHEMA}")
print(f"Openflow User: {OPENFLOW_USER}")
print(f"Deployment: {DEPLOYMENT_NAME}")
print(f"Warehouse: {WAREHOUSE_NAME}")
print(f"Ingress Network Rule: {INGRESS_NETWORK_RULE_NAME}")
print(f"Snowpipe Streaming Network Rule: {SNOWPIPE_STREAMING_NETWORK_RULE_NAME}")
print(f"Snowpipe Streaming EAI: {SNOWPIPE_STREAMING_EAI_NAME}")
print(f"IP Addresses: {IP_ADDRESS_LIST}")
print(f"Runtime Role: {RUNTIME_ROLE_NAME}")
print(f"Event Table: {EVENT_TABLE_NAME}")


## Step 1: Core Snowflake Setup

This step sets up the foundational Snowflake resources required for Openflow SPCS deployment.

**Reference**: [Set up Openflow - Snowflake Deployment: Core Snowflake](https://docs.snowflake.com/en/user-guide/data-integration/openflow/setup-openflow-spcs-sf)

### What we'll create:
1. **OPENFLOW_ADMIN role** - Administrative role for managing Openflow
2. **Required privileges** - Account-level permissions for Openflow operations
3. **Network rule** - Allow ingress from SPCS and your BYOC deployment (if account-level network policy exists)
4. **BCR Bundle 2025_06** - Enable required behavior for integration-level network policies


### Step 1.1: Set Context and Create OPENFLOW_ADMIN Role


In [ ]:
-- Set context and create the OPENFLOW_ADMIN role
USE ROLE {{IMPLEMENTATION_ROLE}};

-- Create database and schema if they don't exist
CREATE DATABASE IF NOT EXISTS {{DATABASE}};
CREATE SCHEMA IF NOT EXISTS {{DATABASE}}.{{SCHEMA}};
USE DATABASE {{DATABASE}};
USE SCHEMA {{SCHEMA}};

-- Create the OPENFLOW_ADMIN role
CREATE ROLE IF NOT EXISTS {{ADMIN_ROLE_NAME}};

-- Grant the role to the specified user
GRANT ROLE {{ADMIN_ROLE_NAME}} TO USER {{OPENFLOW_USER}};

-- Set the default role for the Openflow user as it cannot be ACCOUNTADMIN
ALTER USER {{OPENFLOW_USER}} SET DEFAULT_ROLE = '{{ADMIN_ROLE_NAME}}';

### Step 1.2: Grant Required Privileges


In [ ]:
-- Grant required Openflow privileges to OPENFLOW_ADMIN role
GRANT CREATE OPENFLOW DATA PLANE INTEGRATION ON ACCOUNT TO ROLE {{ADMIN_ROLE_NAME}};
GRANT CREATE OPENFLOW RUNTIME INTEGRATION ON ACCOUNT TO ROLE {{ADMIN_ROLE_NAME}};

-- Compute pools is required for Openflow Deployments
GRANT CREATE COMPUTE POOL ON ACCOUNT TO ROLE {{ADMIN_ROLE_NAME}};

-- We want the Openflow Admin to create and own the Runtime Roles later on
GRANT CREATE ROLE ON ACCOUNT TO ROLE {{ADMIN_ROLE_NAME}};

-- We also want the Admin to create and use the event table later
GRANT USAGE ON DATABASE {{DATABASE}} TO ROLE {{ADMIN_ROLE_NAME}};
GRANT USAGE ON SCHEMA {{DATABASE}}.{{SCHEMA}} TO ROLE {{ADMIN_ROLE_NAME}};
GRANT CREATE EVENT TABLE ON SCHEMA {{DATABASE}}.{{SCHEMA}} TO ROLE {{ADMIN_ROLE_NAME}};

### Step 1.3: Create Network Rule (Only if Account-Level Network Policy Exists)

**Note**: This step is only required if you have an active account-level network policy. Check in Snowsight under **Governance & Security** > **Network Policies** for policies with status "Active on account".


In [ ]:
-- OPTIONAL Create network rule for Openflow SPCS ingress (only if account network policy exists)
-- Create the network rule using the configuration from ingressPersistenceTask.ipynb
CREATE OR REPLACE NETWORK RULE {{INGRESS_NETWORK_RULE_NAME}}
   MODE = INGRESS
   TYPE = IPV4
   VALUE_LIST = {{IP_ADDRESS_LIST}}
   COMMENT = 'Allow ingress from Openflow SPCS (10.16.0.0/12) and BYOC deployment';

-- Add the network rule to your account-level network policy (uncomment and update if needed)
-- ALTER NETWORK POLICY {{NETWORK_POLICY_NAME}} ADD ALLOWED_NETWORK_RULE_LIST = ({{NETWORK_RULE_NAME}});

#### Apply the Rule to your Network Policy if you have one

**IMPORTANT** The SET operator here will override the Network Policy Rule List. If you have existing rules you should add this Ingress rule to them, or they will be squashed

In [ ]:
-- By default, you should add the new rule to the existing list
ALTER NETWORK POLICY {{NETWORK_POLICY_NAME}} ADD ALLOWED_NETWORK_RULE_LIST= ({{INGRESS_NETWORK_RULE_NAME}});

-- Alternatively, you can set the entire list at once, but it will remove all other rules
-- ALTER NETWORK POLICY {{NETWORK_POLICY_NAME}} SET ALLOWED_NETWORK_RULE_LIST = ({{INGRESS_NETWORK_RULE_NAME}});

### Step 1.4: Enable BCR Bundle 2025_06

This bundle is required for Database CDC, SaaS, Streaming, or Slack connectors to ensure connectivity to Snowpipe Streaming.


In [ ]:
-- Check and enable BCR Bundle 2025_06
-- Check the current status of the bundle
CALL SYSTEM$BEHAVIOR_CHANGE_BUNDLE_STATUS('2025_06');

In [ ]:
-- Enable the bundle if it's disabled
CALL SYSTEM$ENABLE_BEHAVIOR_CHANGE_BUNDLE('2025_06');

## Step 2: Create Openflow Deployment

This step creates the Openflow deployment environment with integrated event logging.

**Reference**: [Set up Openflow - Snowflake Deployment: Create deployment](https://docs.snowflake.com/en/user-guide/data-integration/openflow/setup-openflow-spcs-deployment)

### What we'll do:
1. **Create Deployment via UI** - Use Snowsight to create the deployment  
2. **Create and Configure Event Table** - Essential for logging, created and applied immediately


In [ ]:
-- Display key variables for deployment creation
SELECT
    '{{ADMIN_ROLE_NAME}}' AS ADMIN_ROLE_NAME,
    '{{DEPLOYMENT_NAME}}' AS DEPLOYMENT_NAME;

### Step 2.1: Launch Openflow and Create Deployment

**Important**: The deployment creation must be done through the Snowsight web interface. Follow the official process:

**Reference**: [Create a deployment](https://docs.snowflake.com/en/user-guide/data-integration/openflow/setup-openflow-spcs-deployment#create-a-deployment)

Follow these steps:

1. **Navigate to Openflow**:
   - Log into Snowsight with your `{{ADMIN_ROLE_NAME}}` role
   - Go to **Data** > **Openflow** in the left navigation

2. **Launch Openflow**:
   - Click **Launch Openflow** button to open the Openflow interface

3. **Create New Deployment**:
   - In the Openflow UI, go to the **Deployments** tab
   - click **Create a deployment**
   - click **Next** on Prerequisites
   - Enter deployment name: `{{DEPLOYMENT_NAME}}`
   - Optionally you may supply additional roles here
   - click **Create Deployment**

4. **Complete Creation**:
   - You will get a popup notifying the wait time, click **Close**
   - Wait for deployment status to show "Active"


### Step 2.2: Create Event Table and Link to Deployment

Now that the deployment exists, create the event table and link it to the deployment for logging.


In [ ]:
-- Create event table for Openflow logging
USE ROLE {{ADMIN_ROLE_NAME}};

CREATE EVENT TABLE IF NOT EXISTS {{DATABASE}}.{{SCHEMA}}.{{EVENT_TABLE_NAME}}
  COMMENT = 'Event table for Openflow deployment logging and monitoring';

-- Verify the event table was created
DESCRIBE EVENT TABLE {{EVENT_TABLE_NAME}};

In [ ]:
-- Link event table to deployment
-- First, get the deployment UUID
SHOW OPENFLOW DATA PLANE INTEGRATIONS;

-- Pick your Deployment from the list. It's probably the most recent.
-- If you're not sure, you can get the UUID in the UI from the deployment details

In [ ]:
-- Then set the event table (replace the UUID with the one from above)
ALTER OPENFLOW DATA PLANE INTEGRATION OPENFLOW_DATAPLANE_5FAC6F12_3054_4AF5_BDF0_4B4306E29EFB
SET EVENT_TABLE = '{{DATABASE}}.{{SCHEMA}}.{{EVENT_TABLE_NAME}}';

## Step 3: Create Runtime Role

This step creates a dedicated role for runtime operations with the necessary privileges.

**Reference**: [Set up Openflow - Snowflake Deployment: Create Runtime role](https://docs.snowflake.com/en/user-guide/data-integration/openflow/setup-openflow-spcs-create-rr)

### What we'll create:
1. **Runtime Role** - Dedicated role for runtime operations
2. **Required Privileges** - Permissions needed for runtime execution
3. **Account Objects** - A warehouse and EAI for accessing necessary services


### Step 3.1: Create Runtime Role


In [ ]:
-- Create runtime role for Openflow operations
USE ROLE {{ADMIN_ROLE_NAME}};
-- Create the runtime role
CREATE ROLE IF NOT EXISTS {{RUNTIME_ROLE_NAME}};

-- Grant the runtime role to OPENFLOW_ADMIN for management
GRANT ROLE {{RUNTIME_ROLE_NAME}} TO ROLE {{ADMIN_ROLE_NAME}};

-- Grant the runtime role to the Openflow user
GRANT ROLE {{RUNTIME_ROLE_NAME}} TO USER {{OPENFLOW_USER}};

### Step 3.2: Create a Warehouse for Openflow Connectors that need one

In [ ]:
CREATE OR REPLACE WAREHOUSE {{WAREHOUSE_NAME}}
     WITH
         WAREHOUSE_SIZE = 'XSMALL'
         AUTO_SUSPEND = 120
         AUTO_RESUME = TRUE;

### Step 3.3: Grant Runtime Privileges

Grant the necessary privileges for runtime operations including database access and compute pool usage.


In [ ]:
-- Grant runtime privileges
-- Database and schema access
GRANT USAGE ON DATABASE {{DATABASE}} TO ROLE {{RUNTIME_ROLE_NAME}};
GRANT USAGE ON SCHEMA {{DATABASE}}.{{SCHEMA}} TO ROLE {{RUNTIME_ROLE_NAME}};

-- Warehouse usage for runtime operations
GRANT USAGE, OPERATE ON WAREHOUSE {{WAREHOUSE_NAME}} TO ROLE {{RUNTIME_ROLE_NAME}};

-- Table creation and management privileges
GRANT CREATE TABLE ON SCHEMA {{DATABASE}}.{{SCHEMA}} TO ROLE {{RUNTIME_ROLE_NAME}};
GRANT CREATE VIEW ON SCHEMA {{DATABASE}}.{{SCHEMA}} TO ROLE {{RUNTIME_ROLE_NAME}};
GRANT CREATE STAGE ON SCHEMA {{DATABASE}}.{{SCHEMA}} TO ROLE {{RUNTIME_ROLE_NAME}};

-- Event table access for logging
GRANT INSERT ON EVENT TABLE {{DATABASE}}.{{SCHEMA}}.{{EVENT_TABLE_NAME}} TO ROLE {{RUNTIME_ROLE_NAME}};
GRANT SELECT ON EVENT TABLE {{DATABASE}}.{{SCHEMA}}.{{EVENT_TABLE_NAME}} TO ROLE {{RUNTIME_ROLE_NAME}};

### Step 3.4: Grant Access to Existing External Access Integrations

If you have existing External Access Integrations (EAIs) for connectors, databases, or external APIs, grant access to the runtime role so they can be used in your runtime.


In [ ]:
-- List all existing External Access Integrations in the account
SHOW EXTERNAL ACCESS INTEGRATIONS;

-- This will show you all available EAIs that you might want to grant to your runtime role

In [ ]:
-- Grant USAGE on relevant EAIs to the runtime role
-- Uncomment and modify the lines below for EAIs you want to use in your runtime

-- Examples of common EAI grants:
-- GRANT USAGE ON INTEGRATION <YOUR_DATABASE_EAI> TO ROLE {{RUNTIME_ROLE_NAME}};
-- GRANT USAGE ON INTEGRATION <YOUR_API_EAI> TO ROLE {{RUNTIME_ROLE_NAME}};
-- GRANT USAGE ON INTEGRATION <YOUR_CONNECTOR_EAI> TO ROLE {{RUNTIME_ROLE_NAME}};

-- The Snowpipe Streaming EAI we created above:
-- GRANT USAGE ON INTEGRATION OPENFLOW_SNOWPIPE_STREAMING_EAI TO ROLE {{RUNTIME_ROLE_NAME}};

-- Verify grants
SHOW GRANTS TO ROLE {{RUNTIME_ROLE_NAME}};

## Step 4: Create Runtime

This step creates the compute pool and runtime environment where Openflow data flows will execute.

**Reference**: [Set up Openflow - Snowflake Deployment: Create Runtime](https://docs.snowflake.com/en/user-guide/data-integration/openflow/setup-openflow-spcs-create-runtime)

### What we'll create:
2. **Runtime** - The execution environment for Openflow operations
3. **Runtime Configuration** - Size and scaling settings


In [ ]:
-- Display key variables for runtime creation
SELECT
    '{{RUNTIME_NAME}}' AS RUNTIME_NAME;

### Step 4.1: Create Runtime via Snowsight UI

**Important**: Runtime creation is done through the Snowsight web interface. Follow these steps:

1. **Navigate to Deployment**:
   - In Snowsight, go to **Data** > **Openflow**
   - Click **Launch Openflow** 
   - Click on the **Runtimes** tab

2. **Create New Runtime**:
   - Click **+ Create Runtime** button
   - Select your Deployment
   - Enter runtime name: `{{RUNTIME_NAME}}`
   - Select Node Type: Medium is recommended
   - Select Min Nodes: 1 is recommended
   - Select Max Noddes: 3 is recommended
   - Select your Runtime Role created earlier
   - Select your EAIs you created earlier

**NOTE: If your Runtime Role is not in the list, refresh or reload the Control Plane tab**

**NOTE: Any EAIs you select must already be GRANTed to the Runtime Role**

3. **Complete Creation**:
   - Review all settings
   - Click **Create** to finalize the runtime

Wait for the Runtime creation to complete.

You may then Click on the Runtime name to launch it, or browse the Connector catalog.

# Step 5: Remove Openflow
If you want to strip out the changes made to the account, you can run this cell.

**IMPORTANT** This will unset the network rule, if you have other network rules in play ensure you only remove the Openflow ones

In [ ]:
-- Step 1: Stop and drop tasks that depend on database objects
USE ROLE {{IMPLEMENTATION_ROLE}};
USE DATABASE {{DATABASE}};
USE SCHEMA {{SCHEMA}};

-- Step 2: Remove network rules from account-level network policy (if they were added)
-- This is required before dropping the database to avoid policy association errors
-- IMPORTANT: This will unset the Network Policy Rule List. If you have other rules in play ensure you only remove the Openflow ones
ALTER NETWORK POLICY {{NETWORK_POLICY_NAME}} UNSET ALLOWED_NETWORK_RULE_LIST;

-- Step 3: Drop account-level security objects (integrations and roles)
DROP ROLE IF EXISTS {{RUNTIME_ROLE_NAME}};
ALTER USER {{OPENFLOW_USER}} SET DEFAULT_ROLE = '{{IMPLEMENTATION_ROLE}}';
DROP ROLE IF EXISTS {{ADMIN_ROLE_NAME}};

-- Step 4: Drop Warehouse
DROP WAREHOUSE IF EXISTS {{WAREHOUSE_NAME}};

-- Step 5: Drop the entire database (removes all child objects automatically)
-- This includes: network rules, event tables, stages, and all other database objects
DROP DATABASE IF EXISTS {{DATABASE}};